Copyright (c) Meta Platforms, Inc. and affiliates.
This software may be used and distributed according to the terms of the Llama 2 Community License Agreement.

## Quick Start Notebook

This notebook shows how to train a Llama 2 model on a single GPU (e.g. A10 with 24GB) using int8 quantization and LoRA.

### Step 0: Install pre-requirements and convert checkpoint

The example uses the Hugging Face trainer and model which means that the checkpoint has to be converted from its original format into the dedicated Hugging Face format.
The conversion can be achieved by running the `convert_llama_weights_to_hf.py` script provided with the transformer package.
Given that the original checkpoint resides under `models/7B` we can install all requirements and convert the checkpoint with:

In [1]:
# %%bash
# pip install llama-recipes transformers datasets accelerate sentencepiece protobuf==3.20 py7zr scipy peft bitsandbytes fire torch_tb_profiler ipywidgets
# TRANSFORM=`python -c "import transformers;print('/'.join(transformers.__file__.split('/')[:-1])+'/models/llama/convert_llama_weights_to_hf.py')"`
# python ${TRANSFORM} --input_dir models --model_size 7B --output_dir models_hf/7B

### Step 0: RAG & DataPreparation 

In [1]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 18.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 258.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 231.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.7/521.7 kB 428.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 235.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 383.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 399.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
%pip install --upgrade --quiet  "unstructured[all-docs]"

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [11]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [16]:
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 6.8 MB/s eta 0:00:00


In [ ]:
### Documents Loader

In [7]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, TextLoader

DOCS_DIR = './docs'
loader = DirectoryLoader(DOCS_DIR, glob='**/*.txt', show_progress=True, loader_cls=TextLoader)
docs = loader.load()
len(docs)


100%|██████████| 29/29 [00:00<00:00, 1891.18it/s]


29

In [ ]:
### Split Files into chunks

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
len(splits)

1363

In [17]:
# Embed
from chromadb.utils import embedding_functions
from langchain_community.embeddings.huggingface import (HuggingFaceEmbeddings)

embeddings_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)

vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=embeddings)

retriever = vectorstore.as_retriever()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [34]:
import pandas as pd    
qa = pd.read_json(path_or_buf='QAndA_JSONL.txt', lines=True)
qa.head()

,input,output
0,"How do China's AI regulations, particularly o...",China's AI regulations prioritize information ...
1,Why should international discourse take China'...,China's AI regulations reshape global AI deplo...
2,What are the structural similarities found in ...,China's AI regulations share three structural ...
3,How does China formulate AI governance regulat...,China formulates AI governance regulations thr...
4,What is the trajectory of Chinese AI governanc...,Chinese AI governance is heading towards draft...


In [36]:
#Retrieve 4 Relevant documents chunks - Feel free to change
#results = vectorstore.similarity_search(query, k=4)
qa['context'] = qa['input'].apply(lambda x: ' '.join([result.page_content for result in vectorstore.similarity_search(x, k=4)]))
qa.head()

,input,output,context
0,"How do China's AI regulations, particularly o...",China's AI regulations prioritize information ...,China’s three most concrete and impactful regu...
1,Why should international discourse take China'...,China's AI regulations reshape global AI deplo...,But international discourse on Chinese AI gove...
2,What are the structural similarities found in ...,China's AI regulations share three structural ...,2 AI legislation in China And despite China’s ...
3,How does China formulate AI governance regulat...,China formulates AI governance regulations thr...,How China Sets AI Governance Policy\nThis pape...
4,What is the trajectory of Chinese AI governanc...,Chinese AI governance is heading towards draft...,"In this series of three papers, I will attempt..."


In [37]:
processed_qa = qa
processed_qa.to_json('qa_with_context.jsonl', orient='records', lines=True)

### Step 1: Load the model

Point base_model to model weight folder

%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

logging.set_verbosity(logging.ERROR)
base_model = "./models_hf/7B"
guanaco_dataset = "mlabonne/guanaco-llama2-1k"
finetuned_model = './finetuned/policy-llama2-7b'

dataset = Dataset(pa.Table.from_pandas(pd.read_json(path_or_buf='QAndA_JSONL.txt', lines=True)))
#dataset = load_dataset(guanaco_dataset, split="train")

In [3]:
dataset[0]

{'input': " How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?",
 'output': 'China\'s AI regulations prioritize information control through measures like barring excessive price discrimination in recommendation algorithms, requiring labels on synthetically generated content in deep synthesis, and demanding "true and accurate" data and outputs in generative AI.'}

In [4]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    #device_map={"": 0, "":1}
    device_map='auto'
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:836: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### Step 2: Load LLaMA tokenizer


In [6]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [7]:
eval_prompt = """
Answer the following question:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
---
Answer:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


Answer the following question:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
---
Answer:

China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control by requiring companies to obtain approval from the government before using these technologies. This is done to ensure that the information being used is accurate and not misleading, and to prevent the spread of false or harmful information.

For example, the Chinese government has implemented regulations that require companies to obtain approval before using recommendation algorithms to suggest products


In [ ]:
#With Context
eval_prompt = """
Answer the following question with the given context with succinct summary:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
Context: China\u2019s three most concrete and impactful regulations on algorithms and AI are its 2021\nregulation on recommendation algorithms, the 2022 rules for deep synthesis (synthetically generated content), and the 2023 draft rules on generative AI. Information control is a central goal of all three measures, but they also contain many other notable provisions. The rules for recommendation algorithms bar excessive price discrimination and protect the rights of workers subject to algorithmic scheduling. \nThe deep synthesis regulation requires conspicuous labels be placed on synthetically generated content. And the draft generative AI regulation requires both the training data and model outputs to be \u201ctrue and accurate,\u201d Summary: China is in the midst of rolling out some of the world\u2019s earliest and most detailed regulations governing artificial intelligence (AI). These include measures governing recommendation algorithms\u2014the most omnipresent form of AI deployed on the internet\u2014as well as new rules for synthetically generated images and chatbots in the mold of ChatGPT. \nChina\u2019s emerging AI governance framework will reshape how the technology is built and deployed within China and internationally, impacting both Chinese technology exports and global AI research networks.\nBut in the West, China\u2019s regulations are often dismissed as irrelevant or seen purely through the lens of a geopolitical competition to write the rules for AI. These extremely demanding requirements for generative AI systems have kicked off a particularly active public debate on the draft regulation. At the time of writing, Chinese scholars, companies, and policymakers are actively discussing how to maintain effective content controls without squashing China\u2019s nascent generative AI industry. The third paper in this series will dive deep into how this policy debate is playing out in public workshops, academic writing, and corporate lobbying.\nCountries and cultures may differ on the specific content of AI regulations, but they can learn from the content-agnostic structure of the regulations themselves. The above Chinese regulations share three structural similarities: the choice of algorithms as a point of entry; the building of regulatory tools and bureaucratic know-how; and the vertical and iterative approach that is laying the groundwork for a capstone AI law. Three regulations require the deepest analysis: recommendation algorithms, \u201cdeep synthesis,\u201d and generative AI. These interconnected documents contain the most targeted and impactful regulations to date, creating concrete requirements for how algorithms and AI are built and deployed in China. Below is a brief overview of each regulation. The remainder of this paper and subsequent papers will expand on the intellectual roots and key bureaucratic actors behind these regulations.\nProvisions on the Management of Algorithmic Recommendations in Internet Information Services
---
Answer:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))

### Step 3: Load LoRA configuration

In [8]:
peft_params = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.05,
    r=8,
    bias="none",
    target_modules = ["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
)

### Step 4: Set training parameters

In [10]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=50,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

### Step 5: Set supervised fine-tuning parameters

In [11]:
def formatting_prompts_func(example) -> list:
    output_texts = []
    for i in range(len(example['input'])):
        text = f"### Question: {example['input'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

In [12]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    #dataset_text_field="input",
    formatting_func=formatting_prompts_func,
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

Map:   0%|          | 0/111 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [13]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Mon Feb 26 18:43:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:3B:00.0 Off |                    0 |
| N/A   35C    P0              56W / 300W |   5841MiB / 32768MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Step 6: Train Model


In [14]:
torch.cuda.empty_cache() 
trainer.train()

{'loss': 2.0851, 'grad_norm': 1.0927106142044067, 'learning_rate': 0.0002, 'epoch': 0.89}


/jet/home/achoudh2/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf/7B - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 1.7365, 'grad_norm': 1.482894778251648, 'learning_rate': 0.0002, 'epoch': 1.79}


/jet/home/achoudh2/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf/7B - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 1.505, 'grad_norm': 1.6199697256088257, 'learning_rate': 0.0002, 'epoch': 2.68}


...

/jet/home/achoudh2/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf/7B - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 0.0789, 'grad_norm': 0.30372029542922974, 'learning_rate': 0.0002, 'epoch': 50.0}


/jet/home/achoudh2/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf/7B - will assume that the vocabulary was not modified.
  warnings.warn(


{'train_runtime': 607.0182, 'train_samples_per_second': 9.143, 'train_steps_per_second': 2.306, 'train_loss': 0.26918484849589214, 'epoch': 50.0}


TrainOutput(global_step=1400, training_loss=0.26918484849589214, metrics={'train_runtime': 607.0182, 'train_samples_per_second': 9.143, 'train_steps_per_second': 2.306, 'train_loss': 0.26918484849589214, 'epoch': 50.0})

In [16]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Mon Feb 26 18:55:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:3B:00.0 Off |                    0 |
| N/A   36C    P0              55W / 300W |  32285MiB / 32768MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
# Save trained model
trainer.model.save_pretrained(finetuned_model)
trainer.tokenizer.save_pretrained(finetuned_model)

/jet/home/achoudh2/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf/7B - will assume that the vocabulary was not modified.
  warnings.warn(


('./finetuned/policy-llama2-7b/tokenizer_config.json',
 './finetuned/policy-llama2-7b/special_tokens_map.json',
 './finetuned/policy-llama2-7b/tokenizer.model',
 './finetuned/policy-llama2-7b/added_tokens.json',
 './finetuned/policy-llama2-7b/tokenizer.json')

In [23]:
eval_prompt = """
Answer the following question:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
---
Answer:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


Answer the following question:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
---
Answer:
China's AI regulations prioritize information control through measures like barring excessive price discrimination in recommendation algorithms, requiring labels on synthetically generated content in deep synthesis, and demanding "true and accurate" data and outputs in generative AI. The goals of these measures are to influence the flow of information in China, with the potential to affect global audiences as well.

## See More

- China AI Regulation Tracker


In [ ]:
%load_ext tensorboard
from tensorboard import notebook
log_dir = "results/runs"
notebook.start("--logdir {} --port 4000".format(log_dir))

Reusing TensorBoard on port 4000 (pid 75313), started 0:01:45 ago. (Use '!kill 75313' to kill it.)